In [2]:
#Import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import boto3
import sagemaker
import json
import random
import io
from sagemaker.session import Session
from sagemaker.s3 import S3Downloader, S3Uploader
import time
from time import gmtime, strftime, sleep
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sklearn import tree, metrics
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime, timedelta
from sagemaker import get_execution_role, image_uris
from sagemaker.clarify import (
    BiasConfig,
    DataConfig,
    ModelConfig,
    ModelPredictedLabelConfig,
    SHAPConfig,
)
from sagemaker.model import Model
from sagemaker.model_monitor import (
    BiasAnalysisConfig,
    CronExpressionGenerator,
    DataCaptureConfig,
    EndpointInput,
    ExplainabilityAnalysisConfig,
    ModelBiasMonitor,
    ModelExplainabilityMonitor,
)

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import ModelQualityMonitor, EndpointInput, CronExpressionGenerator
from sagemaker.model_monitor.dataset_format import DatasetFormat
from threading import Thread

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Model Quality Monitor

In [ ]:
%store -r bucket
%store -r region
%store -r prefix
%store -r job_name

In [15]:
bucket = Session().default_bucket()
region = boto3.Session().region_name
s3_client = boto3.client("s3", region_name=region)
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

image = sagemaker.image_uris.retrieve(
    framework="xgboost", region=boto3.Session().region_name, version="1.7-1"
)

In [17]:
#Create prefixes
s3_key = f"s3://{bucket}/{prefix}"
s3_capture_upload_path = f"{s3_key}/datacapture"
ground_truth_upload_path = f"{s3_key}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
s3_report_path = f"{s3_key}/reports"

print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

baseline_results_uri = f"{s3_key}/baselining"
print(f"Baseline results uri: {baseline_results_uri}")

endpoint_instance_count = 1
endpoint_instance_type = "ml.m4.xlarge"
schedule_expression = CronExpressionGenerator.hourly()

Capture path: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/datacapture
Ground truth path: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/ground_truth_data/2024-06-23-02-03-21
Report path: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/reports
Baseline results uri: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining


In [18]:
#Define model and files 
model_url = "s3://{}/{}/output/{}/{}/output/model.tar.gz".format(bucket, prefix, job_name, job_name)
# test_dataset = "test_data/test-dataset-input-cols.csv"
validation_dataset = "validation_data_with_header.csv"
dataset_type = "text/csv"

with open(validation_dataset) as f:
    headers_line = f.readline().rstrip()
all_headers = headers_line.split(",")
label_header = all_headers[0]

In [19]:
#Create a model from the file on S3
model_name = f"bids-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("Model name: ", model_name)
endpoint_name = f"bids-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("Endpoint name: ", endpoint_name)

Model name:  bids-model-monitor-2024-06-23-0203
Endpoint name:  bids-model-monitor-2024-06-23-0203


In [20]:
#Deploy with data capture enabled
model = Model(
    role=role,
    name=model_name,
    image_uri=image,
    model_data=model_url,
    sagemaker_session=sess,
)

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
)
print(f"Deploying model {model_name} to endpoint {endpoint_name}")
model.deploy(
    initial_instance_count=endpoint_instance_count,
    instance_type=endpoint_instance_type,
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

Deploying model bids-model-monitor-2024-06-23-0203 to endpoint bids-model-monitor-2024-06-23-0203
-------!

In [21]:
#Create predictor object
predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sess, serializer=CSVSerializer()
)

In [22]:
#Create baseline with 0.8 as the cut off value
bids_cutoff = 0.8
validate_dataset = "validation_data_predictions.csv"

In [23]:
limit = 200
i = 0
with open(f"{validate_dataset}", "w") as baseline_file:
    baseline_file.write("probability,prediction,label\n")  # our header
    with open("validation_data.csv", "r") as f:
        for row in f:
            (label, input_cols) = row.split(",", 1)
            probability = float(predictor.predict(input_cols))
            prediction = "1" if probability > bids_cutoff else "0"
            baseline_file.write(f"{probability},{prediction},{label}\n")
            i += 1
            if i > limit:
                break
            print(".", end="", flush=True)
            sleep(0.5)
print()
print("Done!")

...........................................................................................................................................................................................
Done!


In [24]:
#View predicitons
!head validation_data_predictions.csv

probability,prediction,label
-0.0003505268250592053,0,0
0.0020377596374601126,0,0
0.0037524779327213764,0,0
0.3750445246696472,0,1
0.05720706656575203,0,0
0.04872201010584831,0,0
0.010198176838457584,0,0
0.016347570344805717,0,0
0.016347570344805717,0,0


In [25]:
#Upload predictions as a baseline
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/data
Baseline results uri: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/results


In [26]:
baseline_dataset_uri = S3Uploader.upload(f"{validate_dataset}", baseline_data_uri)
baseline_dataset_uri

's3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/data/validation_data_predictions.csv'

In [27]:
# Create the model quality monitoring object
bids_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sess,
)

In [28]:
# Name of the model quality baseline job
baseline_job_name = f"xgb-bids-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [29]:
# Execute the baseline suggestion job.
job = bids_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    inference_attribute="prediction",
    probability_attribute="probability",
    ground_truth_attribute="label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name xgb-bids-model-baseline-job-2024-06-23-0210


...........................................................!

In [30]:
baseline_job = bids_model_quality_monitor.latest_baselining_job

In [31]:
#See the baseline statistics 
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T

,0
confusion_matrix.0.0,177
confusion_matrix.0.1,1
confusion_matrix.1.0,9
confusion_matrix.1.1,0
recall.value,0.0
recall.standard_deviation,None
precision.value,0.0
precision.standard_deviation,None
accuracy.value,0.946524
accuracy.standard_deviation,None


In [32]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.0,LessThanThreshold
precision,0.0,LessThanThreshold
accuracy,0.946524,LessThanThreshold
true_positive_rate,0.0,LessThanThreshold
true_negative_rate,0.994382,LessThanThreshold
false_positive_rate,0.005618,GreaterThanThreshold
false_negative_rate,1.0,GreaterThanThreshold
auc,0.743446,LessThanThreshold
f0_5,0.0,LessThanThreshold
f1,0.0,LessThanThreshold


In [33]:
#Generate prediction data
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = sess.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test_data_noID.csv")
        except sess.sagemaker_runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever)
thread.start()

In [34]:
#Generate synthetic ground truth
def ground_truth_with_id(inference_id):
    random.seed(inference_id)  # to get consistent results
    rand = random.random()
    return {
        "groundTruthData": {
            "data": "1" if rand < 0.7 else "0",  # randomly generate positive labels 70% of the time
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }


def upload_ground_truth(records, upload_time):
    fake_records = [json.dumps(r) for r in records]
    data_to_upload = "\n".join(fake_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(fake_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

In [35]:
NUM_GROUND_TRUTH_RECORDS = 334  # 334 are the number of rows in data we're sending for inference
def generate_fake_ground_truth_forever():
    j = 0
    while True:
        fake_records = [ground_truth_with_id(i) for i in range(NUM_GROUND_TRUTH_RECORDS)]
        upload_ground_truth(fake_records, datetime.utcnow())
        j = (j + 1) % 5
        sleep(60 * 60)  # do this once an hour


gt_thread = Thread(target=generate_fake_ground_truth_forever)
gt_thread.start()

Uploading 334 records to s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/ground_truth_data/2024-06-23-02-03-21/2024/06/23/02/1707.jsonl
Uploading 334 records to s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/ground_truth_data/2024-06-23-02-03-21/2024/06/23/03/1707.jsonl
Uploading 334 records to s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/ground_truth_data/2024-06-23-02-03-21/2024/06/23/04/1708.jsonl
Uploading 334 records to s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/ground_truth_data/2024-06-23-02-03-21/2024/06/23/05/1708.jsonl


In [36]:
##Monitoring schedule name
bids_monitor_schedule_name = (
    f"xgb-bids-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}"
)

In [37]:
# Create an enpointInput
endpointInput = EndpointInput(
    endpoint_name=predictor.endpoint_name,
    probability_attribute="0",
    probability_threshold_attribute=0.5,
    destination="/opt/ml/processing/input_data",
)

In [38]:
# Create the monitoring schedule to execute every hour.
response = bids_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=bids_monitor_schedule_name,
    endpoint_input=endpointInput,
    output_s3_uri=baseline_results_uri,
    problem_type="BinaryClassification",
    ground_truth_input=ground_truth_upload_path,
    constraints=baseline_job.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: xgb-bids-monitoring-schedule-2024-06-23-0217


In [39]:
# Create the monitoring schedule
# You will see the monitoring schedule in the 'Scheduled' status
bids_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:215730597255:monitoring-schedule/xgb-bids-monitoring-schedule-2024-06-23-0217',
 'MonitoringScheduleName': 'xgb-bids-monitoring-schedule-2024-06-23-0217',
 'MonitoringScheduleStatus': 'Scheduled',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 6, 23, 2, 17, 11, 3000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 6, 23, 2, 17, 15, 437000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-06-23-02-17-10-384',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'bids-model-monitor-2024-06-23-0203',
 'ResponseMetadata': {'RequestId': '7b004a95-b125-423f-8e7c-996bd2fb64fa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b004a95-b125-423f-8e7c-996bd2fb64fa',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '602',
   'date':

In [40]:
# Initially there will be no executions since the first execution happens at the top of the hour
# Note that it is common for the execution to luanch upto 20 min after the hour.
executions = bids_model_quality_monitor.list_executions()
executions

[]

In [41]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = bids_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution.........................................................................................................................................................................................................................................................................................................
Execution found!


In [42]:
while not executions:
    executions = bids_model_quality_monitor.list_executions()
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

{'ProcessingInputs': [{'InputName': 'constraints',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/results/constraints.json',
    'LocalPath': '/opt/ml/processing/baseline/constraints',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/results/merge/bids-model-monitor-2024-06-23-0203/AllTraffic/2024/06/23/02',
    'LocalPath': '/opt/ml/processing/input_data/bids-model-monitor-2024-06-23-0203/AllTraffic/2024/06/23/02',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'result',
    'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-215730597255/sagemaker-features

In [43]:
status = execution["MonitoringExecutionStatus"]

while status in ["Pending", "InProgress"]:
    print("Waiting for execution to finish", end="")
    latest_execution.wait(logs=False)
    latest_job = latest_execution.describe()
    print()
    print(f"{latest_job['ProcessingJobName']} job status:", latest_job["ProcessingJobStatus"])
    print(
        f"{latest_job['ProcessingJobName']} job exit message, if any:",
        latest_job.get("ExitMessage"),
    )
    print(
        f"{latest_job['ProcessingJobName']} job failure reason, if any:",
        latest_job.get("FailureReason"),
    )
    sleep(
        30
    )  # model quality executions consist of two Processing jobs, wait for second job to start
    latest_execution = bids_model_quality_monitor.list_executions()[-1]
    execution = bids_model_quality_monitor.describe_schedule()["LastMonitoringExecutionSummary"]
    status = execution["MonitoringExecutionStatus"]

print("Execution status is:", status)

if status != "Completed":
    print(execution)
    print(
        "====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures."
    )

Waiting for execution to finish........................................................................!
model-quality-monitoring-202406230300-c2f0fcc92c14d4f6489691a2 job status: Completed
model-quality-monitoring-202406230300-c2f0fcc92c14d4f6489691a2 job exit message, if any: CompletedWithViolations: Job completed successfully with 4 violations.
model-quality-monitoring-202406230300-c2f0fcc92c14d4f6489691a2 job failure reason, if any: None
Execution status is: CompletedWithViolations
{'MonitoringScheduleName': 'xgb-bids-monitoring-schedule-2024-06-23-0217', 'ScheduledTime': datetime.datetime(2024, 6, 23, 3, 0, tzinfo=tzlocal()), 'CreationTime': datetime.datetime(2024, 6, 23, 3, 7, 42, 191000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2024, 6, 23, 3, 26, 14, 800000, tzinfo=tzlocal()), 'MonitoringExecutionStatus': 'CompletedWithViolations', 'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:215730597255:processing-job/model-quality-monitoring-202406230300-c2f0fcc92c14d4f64

In [44]:
latest_execution = bids_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

Report Uri: s3://sagemaker-us-east-1-215730597255/sagemaker-featurestore/baselining/results/bids-model-monitor-2024-06-23-0203/xgb-bids-monitoring-schedule-2024-06-23-0217/2024/06/23/03


In [45]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": bids_monitor_schedule_name},
]

In [46]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

f2_best_constant_classifier
false_negative_rate
recall_best_constant_classifier
f2
false_positive_rate
auc
accuracy_best_constant_classifier
f1
total_number_of_violations
f1_best_constant_classifier
f0_5
true_negative_rate
f0_5_best_constant_classifier
precision
recall
true_positive_rate
precision_best_constant_classifier
au_prc
accuracy


In [47]:
alarm_name = "MODEL_QUALITY_ACCURACY"
alarm_desc = (
    "Trigger an CloudWatch alarm when the accuracy drifts away from the baseline constraints"
)
mdoel_quality_accuracy_threshold = (
    0.93  
)
metric_name = "accuracy"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": bids_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_accuracy_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': '6cc554ae-f82d-40f8-8b9e-7e6e3406c805',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6cc554ae-f82d-40f8-8b9e-7e6e3406c805',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Sun, 23 Jun 2024 03:50:40 GMT'},
  'RetryAttempts': 0}}

In [48]:
alarm_name = "MODEL_QUALITY_PRECISION"
alarm_desc = (
    "Trigger an CloudWatch alarm when the precision drifts away from the baseline constraints"
)
mdoel_quality_precision_threshold = (
    0.5  
)
metric_name = "precision"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": bids_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=6,
    DatapointsToAlarm=1,
    Threshold=mdoel_quality_precision_threshold,
    ComparisonOperator="LessThanOrEqualToThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': '21f2e97f-422c-4378-807f-f750fa987be9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '21f2e97f-422c-4378-807f-f750fa987be9',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Sun, 23 Jun 2024 03:50:43 GMT'},
  'RetryAttempts': 0}}

# Cleanup Resources

In [ ]:
bids_model_quality_monitor.delete_monitoring_schedule()
sleep(60)  # actually wait for the deletion

INFO:sagemaker:Deleting Monitoring Schedule with name: xgb-bids-monitoring-schedule-2024-06-17-2342
INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2024-06-17-23-42-43-829


In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: bids-model-monitor-2024-06-17-2226
INFO:sagemaker:Deleting endpoint configuration with name: bids-model-monitor-2024-06-17-2226
INFO:sagemaker:Deleting endpoint with name: bids-model-monitor-2024-06-17-2226


In [ ]:
invoke_endpoint_thread.terminate()
ground_truth_thread.terminate()

In [ ]:
train_feature_group.delete()
test_feature_group.delete()
sagemaker.delete_endpoint(EndpointName=endpoint_name)